In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [24]:
from llama_index.llms.openai import OpenAI

# llm = OpenAI(model="gpt-3.5-turbo")
llm = OpenAI(model="gpt-4-1106-preview")

In [5]:
papers = ['board_meeting_dec23.pdf',
         'board_meeting_nov23.pdf',
         'board_meeting_oct23.pdf']

In [6]:
from utils import get_doc_tools
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

Getting tools for paper: board_meeting_dec23.pdf
Getting tools for paper: board_meeting_nov23.pdf
Getting tools for paper: board_meeting_oct23.pdf


In [8]:
all_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

# Agents with tool selection capability

In [9]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [10]:
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [11]:
tools = obj_retriever.retrieve(
    "summarise the nov 23 board meeting notes"
)

In [13]:
tools[0].metadata

ToolMetadata(description='Useful for summarization questions related to board_meeting_nov23', name='summary_tool_board_meeting_nov23', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)

In [14]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=llm, 
    system_prompt=""" \
You are an agent designed to answer queries over a set of given papers.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    verbose=True
)
agent = AgentRunner(agent_worker)

In [21]:
response = agent.query(
    "was anything discussed about atopic dermatitis in any of the board meeting"
)
# print(str(response))

Added user message to memory: was anything discussed about atopic dermatitis in any of the board meeting
=== Calling Function ===
Calling function: vector_tool_board_meeting_dec23 with args: {"query": "atopic dermatitis"}
=== Function Output ===
The College of Pharmacy recommends updating the approval criteria for Cibinqo ™ (abrocitinib) and Rinvoq® (upadacitinib) for the treatment of atopic dermatitis. The criteria include having an FDA-approved diagnosis of moderate-to-severe atopic dermatitis not adequately controlled with other systemic drug products, documented trials with specific topical therapies, and the medication being prescribed by a dermatologist, allergist, or immunologist. Additionally, there are specific age requirements for each medication and restrictions on concurrent use with certain therapies.
=== Calling Function ===
Calling function: vector_tool_board_meeting_nov23 with args: {"query": "atopic dermatitis"}
=== Function Output ===
Approval criteria for medications

# Agents without tool selection capabaility

In [16]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    all_tools, 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [23]:
response = agent.query(
    "what was discussed about Annual Review of Imcivree during october 2023 board meeting"
    "summarise the discussion"    
)

Added user message to memory: what was discussed about Annual Review of Imcivree during october 2023 board meetingsummarise the discussion
=== Calling Function ===
Calling function: summary_tool_board_meeting_oct23 with args: {"input": "Annual Review of Imcivree"}


Retrying llama_index.llms.openai.base.OpenAI._achat in 0.6758564311753312 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-Xnt90aHtb4ZX1UUscgKKvCl1 on tokens per min (TPM): Limit 60000, Used 57977, Requested 14528. Please try again in 12.505s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.


=== Function Output ===
The annual review of Imcivree includes evaluating the current prior authorization criteria, utilization of Imcivree, prior authorization process for Imcivree, market news and updates related to Imcivree, recommendations from the College of Pharmacy, details on the utilization of Imcivree, an increase in utilization compared to the previous year, demographics of members using Imcivree, top prescriber specialties, and its importance as a treatment option for eligible individuals for chronic weight management in individuals with specific genetic conditions.
=== LLM Response ===
The discussion about the Annual Review of Imcivree during the October 2023 board meeting included evaluating the current prior authorization criteria, utilization of Imcivree, prior authorization process, market news and updates, recommendations from the College of Pharmacy, details on utilization, increased utilization compared to the previous year, demographics of members using Imcivree, t

In [22]:
response = agent.query(
    "was anything discussed about atopic dermatitis in any of the board meeting"
    )

Added user message to memory: was anything discussed about atopic dermatitis in any of the board meeting
=== Calling Function ===
Calling function: vector_tool_board_meeting_oct23 with args: {"query": "atopic dermatitis"}
=== Function Output ===
Unique criteria applies for a diagnosis of atopic dermatitis (AD).
=== Calling Function ===
Calling function: vector_tool_board_meeting_nov23 with args: {"query": "atopic dermatitis"}
=== Function Output ===
Approval criteria for medications used to treat atopic dermatitis include having an FDA-approved diagnosis of moderate-to-severe atopic dermatitis not adequately controlled with topical prescription therapies, being a certain age or older, having documented trial failures with specific topical therapies, being prescribed by a qualified healthcare provider, and in some cases, meeting specific trial or age restriction exceptions.
=== Calling Function ===
Calling function: vector_tool_board_meeting_dec23 with args: {"query": "atopic dermatitis

In [26]:
response = agent.query(
    "was anything discussed about atopic dermatitis in any of the board meeting"
    "give the source nodes for reference"
    )

Added user message to memory: was anything discussed about atopic dermatitis in any of the board meetinggive the source nodes for reference
=== Calling Function ===
Calling function: vector_tool_board_meeting_dec23 with args: {"query": "atopic dermatitis"}
=== Function Output ===
The College of Pharmacy recommends updating the approval criteria for Cibinqo ™ (abrocitinib) and Rinvoq® (upadacitinib) for the treatment of atopic dermatitis. The criteria include an FDA-approved diagnosis of moderate-to-severe atopic dermatitis not adequately controlled with other systemic drug products, documented trials with specific topical therapies, and the need for prescription by a dermatologist, allergist, or immunologist. Additionally, there are specific requirements related to age, prior treatments, concurrent medication use, and maximum approvable doses for Rinvoq®.
=== Calling Function ===
Calling function: vector_tool_board_meeting_nov23 with args: {"query": "atopic dermatitis"}
=== Function Ou